# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [2]:
import findspark
findspark.init('/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('churn').getOrCreate()

In [3]:
df= spark.read.csv('customer_churn.csv', inferSchema = True, header = True)

In [4]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
from pyspark.ml.feature import VectorAssembler

In [12]:
my_cols = ['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites']
assembler = VectorAssembler(inputCols = my_cols, outputCol = 'features')

In [13]:
output = assembler.transform(df)

In [14]:
final_data = output.select('features', 'churn')

In [15]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
lr_churn = LogisticRegression(labelCol='churn')

In [18]:
fitted_churn_model = lr_churn.fit(train_data)

21/11/29 08:07:08 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
21/11/29 08:07:08 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [19]:
training_sum = fitted_churn_model.summary

In [20]:
training_sum.predictions.describe().show()

/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------+------------------+------------------+
|summary|             churn|        prediction|
+-------+------------------+------------------+
|  count|               649|               649|
|   mean|0.1679506933744222|0.1325115562403698|
| stddev|0.3741108273564827|0.3393075878033656|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
pred_label = fitted_churn_model.evaluate(test_data)

In [24]:
pred_label.predictions.show()

/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.56759890285111...|[0.63820895011996...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.43489242826348...|[0.80766246837290...|       0.0|
|[28.0,11204.23,0....|    0|[1.54293215678812...|[0.82389057033141...|       0.0|
|[28.0,11245.38,0....|    0|[3.37160542580723...|[0.96680525249025...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.00738568851272...|[0.98214377793600...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.09174207574037...|[0.98356454021113...|       0.0|
|[29.0,13255.05,1....|    0|[3.86827778426307...|[0.97953331454803...|       0.0|
|[32.0,9472.72,1.0...|    0|[3.75302099602304...|[0.97709035202499...|       0.0|
|[32.0,9885.12,1.0...|    1|[1.71392879333092...|[0.84734517268027...|       0.0|
|[32.0,12479.72,

In [25]:
churn_eval = BinaryClassificationEvaluator(labelCol='churn', rawPredictionCol = 'prediction')

In [27]:
auc = churn_eval.evaluate(pred_label.predictions)

/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [28]:
auc

0.7272357723577236

## Predict on New Data

In [30]:
final_lr_model = lr_churn.fit(final_data) # use all the data to fit

In [31]:
new_customers = spark.read.csv('new_customers.csv', inferSchema = True, header = True)

In [32]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [33]:
test_new_customers = assembler.transform(new_customers)

In [34]:
final_results = final_lr_model.transform(test_new_customers)

In [39]:
final_results.select('Company', 'Prediction').show()

+----------------+----------+
|         Company|Prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

